In [26]:
import carto2gpd
import cenpy
import numpy as np
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
import datetime as dt
import missingno as msno

import hvplot.pandas
import holoviews as hv

from io import BytesIO
import gzip
from urllib.request import urlopen

In [2]:
states = ['pa','il','ga','ca']
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

In [139]:
url = "https://lehd.ces.census.gov/data/lodes/LODES7/pa/wac/pa_wac_S000_JT00_"
wac = dd.concat([pd.read_csv(gzip.open(BytesIO(urlopen(url+year+".csv.gz").read()))).assign(year=year) for year in years])


In [168]:
wac['w_geocode'] = wac['w_geocode'].astype('str').str[:12]
wac.compute()

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,...,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate,year
0,420010301011,15,1,14,0,3,9,3,14,0,...,0,0,0,0,0,0,0,0,20160228,2009
1,420010301011,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,20160228,2009
2,420010301011,54,9,32,13,17,29,8,50,0,...,0,0,0,0,0,0,0,0,20160228,2009
3,420010301011,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,20160228,2009
4,420010301011,25,5,15,5,20,5,0,0,0,...,0,0,0,0,0,0,0,0,20160228,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103737,421330240022,4,1,3,0,3,0,1,0,0,...,0,0,0,0,0,0,0,0,20211018,2019
103738,421330240022,9,2,3,4,3,4,2,0,0,...,0,0,0,0,0,0,0,0,20211018,2019
103739,421330240022,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,20211018,2019
103740,421330240022,65,6,43,16,6,16,43,0,0,...,0,0,0,0,0,0,0,0,20211018,2019


In [153]:
phl_bg = gpd.read_file("bg/tl_2019_42_bg/tl_2019_42_bg.shp").query("COUNTYFP in ['017','029','045','091','101']").to_crs('EPSG:3857')

In [180]:
wac_geo = phl_bg[['GEOID','geometry']].merge(wac.compute(), left_on='GEOID', right_on='w_geocode', how='inner')
wac_all = wac_geo.groupby('GEOID')['C000'].sum().reset_index()

In [181]:
wac_all.hvplot(geo=True,
                c="C000",
                crs=3857,
                hover_fill_color="gray",
                width=600, 
                height=400, 
                cmap='viridis')

:Curve   [index]   (C000)